Note: This notebook about 40 MB of embedded output data, which should be put under version control.   
Please make sure to clear all cell output using the Cell>All Output>Clear command from the menu befor committing changes!  
Use Cell>Run all to reproduce the output.

注意:这个笔记本大约有40mb的嵌入式输出数据，应该放在版本控制下。
在提交更改之前，请确保使用菜单中的单元格> all output > clear命令清除所有单元格输出使用单元>全部运行以重现输出。

# 前期准备

In [1]:
import sys 
sys.path.append(r'your_path')
print(sys.path)

# 这个很有用

['', 'D:\\vs17\\Anaconda3_64\\python36.zip', 'D:\\vs17\\Anaconda3_64\\DLLs', 'D:\\vs17\\Anaconda3_64\\lib', 'D:\\vs17\\Anaconda3_64', 'C:\\Users\\zx305\\AppData\\Roaming\\Python\\Python36\\site-packages', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\win32', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\win32\\lib', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\Pythonwin', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\zx305\\.ipython', 'your_path']


In [ ]:
!pip install msgpack --user

In [ ]:
#!python -m pip install --upgrade pip
#!python3 -m pip install --upgrade pip


In [4]:
!pip install librosa --user

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
!pip install mir_eval --user

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
!pip install pylearn2 --user

  Could not find a version that satisfies the requirement pylearn2 (from versions: )
No matching distribution found for pylearn2
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
# !pip install deepthought --user #这段不是这么安装的！但还没有解决
!pip uninstall deepthought 

In [2]:
!pip list


Package                            Version  
---------------------------------- ---------
alabaster                          0.7.10   
anaconda-client                    1.6.14   
anaconda-navigator                 1.8.7    
anaconda-project                   0.8.2    
asn1crypto                         0.24.0   
astroid                            1.6.3    
astropy                            3.0.2    
attrs                              18.1.0   
audioread                          2.1.8    
Babel                              2.5.3    
backcall                           0.1.0    
backports.shutil-get-terminal-size 1.0.0    
beautifulsoup4                     4.6.0    
bitarray                           0.8.1    
bkcharts                           0.2      
blaze                              0.11.3   
bleach                             2.1.3    
bokeh                              0.12.16  
boto                               2.48.0   
Bottleneck                         1.2.1    
certifi   

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# 程序正式开始段

In [3]:
import sys 
sys.path.append(r'your_path')
print(sys.path)

# 这个很有用

['', 'D:\\vs17\\Anaconda3_64\\python36.zip', 'D:\\vs17\\Anaconda3_64\\DLLs', 'D:\\vs17\\Anaconda3_64\\lib', 'D:\\vs17\\Anaconda3_64', 'C:\\Users\\zx305\\AppData\\Roaming\\Python\\Python36\\site-packages', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\win32', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\win32\\lib', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\Pythonwin', 'D:\\vs17\\Anaconda3_64\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\zx305\\.ipython', 'your_path', 'your_path']


In [4]:
import numpy as np
import librosa #音频处理用的 
import mir_eval
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import IPython.display
from IPython.display import display

import os
import deepthought

from deepthought.deepthought.datasets.openmiir.metadata import load_stimuli_metadata, save_beat_times

STIMULI_VERSION = 2   # change to 1 for older stimuli version
data_root = os.path.join(deepthought.DATA_PATH, 'OpenMIIR')
default_save_beat_times = False # change to True to save beat time to txt file

ModuleNotFoundError: No module named 'deepthought.deepthought'

In [5]:
def play_beats(y, sr, beats):
    
    if y is None:
        # Sonify the beats only
        y_beat = mir_eval.sonify.clicks(beats, sr, length=len(y))
    else:
        # Sonify the beats and add them to y
        y_beat = y + mir_eval.sonify.clicks(beats, sr, length=len(y))    
    
    return IPython.display.Audio(data=y_beat, rate=sr)

def visualize(y, sr, title=None, playback=True, beats=None):
    
    # show playback widget above figure
    if playback:
        if title is not None:
            print title
        
        if beats is None:
            display(IPython.display.Audio(data=y, rate=sr))
        else:
            beat_times = librosa.frames_to_time(beats, sr=sr, hop_length=64)
            display(play_beats(y, sr, beat_times))
    
    # Let's make and display a mel-scaled power (energy-squared) spectrogram
    # We use a small hop length of 64 here so that the frames line up with the beat tracker example below.
    S = librosa.feature.melspectrogram(y, sr=sr, n_fft=2048, hop_length=64, n_mels=128)

    # Convert to log scale (dB). We'll use the peak power as reference.
    log_S = librosa.logamplitude(S, ref_power=np.max)

    # Make a new figure
    plt.figure(figsize=(12,4))

    # Display the spectrogram on a mel scale
    # sample rate and hop length parameters are used to render the time axis
    librosa.display.specshow(log_S, sr=sr, hop_length=64, x_axis='time', y_axis='mel')

    # Put a descriptive title on the plot
    if title is not None:
        plt.title('mel power spectrogram ({})'.format(title))

    if beats is not None:
        # Let's draw lines with a drop shadow on the beat events
        plt.vlines(beats, 0, log_S.shape[0], colors='k', linestyles='-', linewidth=2.5)
        plt.vlines(beats, 0, log_S.shape[0], colors='w', linestyles='-', linewidth=1.5)
        
    # draw a color bar
    plt.colorbar(format='%+02.0f dB')

    # Make the figure layout compact
    plt.tight_layout()

    # This make sure the figures are plotted in place and not after text and audio
    plt.show()
    plt.close()
    

def _analyze_beats(audio_filepath, bpm, label=None, tightness=400, offset=0, duration=None, vy=True, vh=True, vp=True, vb=True):
    print audio_filepath
    # load audio file
#     sr = 22050  # default
    sr = 44100  # slower but gives better results for Harry Potter Theme
    y, sr = librosa.load(audio_filepath, sr=sr, offset=offset, duration=duration)
    
    if label is not None:
        print label
    
    if vy:
        visualize(y, sr, 'original')
    
    # split into harmonic and percussive component
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    
    if vh:
        visualize(y_harmonic, sr, 'harmonic component')
    if vp:
        visualize(y_percussive, sr, 'percussive component')
    
    # Now, let's run the beat tracker
    # We'll use the percussive component for this part
    # By default, the beat tracker will trim away any leading or trailing beats that don't appear strong enough.
    # To disable this behavior, call beat_track() with trim=False.

    tempo, beats = librosa.beat.beat_track(y=y_percussive, sr=sr, hop_length=64, trim=False, start_bpm=bpm, tightness=tightness)

    # Let's re-draw the spectrogram, but this time, overlay the detected beats
    if vb:
        visualize(y, sr, 'with beats', beats=beats)

    print 'Offset:                 %.4f s' % offset
        
    print 'Expected tempo:         %.2f BPM' % bpm
    print 'Estimated tempo:        %.2f BPM' % tempo
    print 'First 5 beat frames:   ', beats[:5]

    # Frame numbers are great and all, but when do those beats occur?
    print 'First 5 beat times:    ', librosa.frames_to_time(beats[:5], sr=sr, hop_length=64)
    
    return tempo, beats, librosa.frames_to_time(beats, sr=sr, hop_length=64)

def get_audio_filepath(meta):
    return os.path.join(data_root, 'audio', 'full.v{}'.format(STIMULI_VERSION), meta['audio_file'])

def analyze_beats(meta, tightness=400, save=default_save_beat_times, **kwargs):
    tempo, beat_frames, beat_times = _analyze_beats(
        audio_filepath=get_audio_filepath(meta),
        label=meta['label'], 
        bpm=meta['bpm'], 
        tightness=tightness, 
        offset=meta['length_of_cue'],
        **kwargs
    )
    
    if save:
        offset = meta['length_of_cue']
        save_beat_times(beat_times, stimulus_id=meta['id'], offset=offset, version=STIMULI_VERSION)

    return tempo, beat_frames, beat_times

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(title)? (<ipython-input-5-fdf3da417a0b>, line 28)

In [ ]:
# NOTE: this is experimental
def analyze_onsets(meta):
    audio_filepath=os.path.join(data_root, 'audio', 'full.v{}'.format(STIMULI_VERSION), meta['audio_file'])
    sr = 44100  # slower but gives better results for Harry Potter Theme
    offset=meta['length_of_cue']
    duration=None
    print sr
    y, sr = librosa.load(audio_filepath, sr=sr, offset=offset, duration=duration)
    """
    # Get onset times from a signal
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr, hop_length=64)
    onset_times = librosa.frames_to_time(onset_frames, sr, hop_length=64)

    # Or use a pre-computed onset envelope
    o_env = librosa.onset.onset_strength(y, sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr, hop_length=64)
    """
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr, hop_length=64)
    print onset_frames
    visualize(y, sr, 'with beats', beats=onset_frames)
    
    o_env = librosa.onset.onset_strength(y, sr=sr)
    plt.plot(o_env)
    onset_frames = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr)
    print onset_frames
    visualize(y, sr, 'with beats', beats=onset_frames*7)

In [ ]:
meta = load_stimuli_metadata(data_root, version=STIMULI_VERSION)
# print meta

# some examples

In [ ]:
# run this to analyze onsets for stimulus 22
analyze_onsets(meta[22])

In [ ]:
# run this to analyze onsets for stimulus 1, specify tightness
tempo, beat_frames, beat_times = analyze_beats(meta[1], tightness=800)
print beat_times
print beat_frames

In [ ]:
# analyze beginning of stimulus 22, 
# suppress visualization of original signal (vy) and harmonic (vh) and percussive (vp) component
_analyze_beats(get_audio_filepath(meta[22]), 
               bpm=166, tightness=250, offset=2.182, duration=4.0, vy=False, vh=False, vp=False);

In [ ]:
# test different tightness settings on stimulus 22
_analyze_beats(get_audio_filepath(meta[22]), 
               bpm=166, tightness=250, offset=0, duration=None, vy=False, vh=False, vp=False);
_analyze_beats(get_audio_filepath(meta[22]), 
               bpm=166, tightness=400, offset=0, duration=None, vy=False, vh=False, vp=False);
_analyze_beats(get_audio_filepath(meta[22]), 
               bpm=166, tightness=800, offset=0, duration=None, vy=False, vh=False, vp=False);

# analyze stimuli

In [ ]:
_ = analyze_beats(meta[1], tightness=1000, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[2], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[3], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[4], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[11], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[12], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[13], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[14], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[21], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[22], tightness=300, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[23], tightness=800, vy=False, vh=False, vp=False)

In [ ]:
_ = analyze_beats(meta[24], tightness=800, vy=False, vh=False, vp=False)

# analyze cue click tracks

In [ ]:
from deepthought.datasets.openmiir.constants import STIMULUS_IDS

for stimulus_id in STIMULUS_IDS:
    tempo, beat_frames, beat_times = _analyze_beats(
        audio_filepath=os.path.join(data_root, 'audio', 'cues.v{}'.format(STIMULI_VERSION), meta[stimulus_id]['cue_file']), 
        label=meta[stimulus_id]['label'], 
        bpm=meta[stimulus_id]['cue_bpm'], 
        tightness=10000, vy=False, vh=False, vp=False
    )
    
    if default_save_beat_times:
        save_beat_times(beat_times, stimulus_id=stimulus_id, cue=True, version=STIMULI_VERSION)